# Import statements
Following packages are necessary:

In [1]:
import sys
sys.path.append("D:\Julian\workspace\endolas\packages")

import random
from endolas import utils
import os
import imageio
import json
from endolas  import closs
from endolas import ccall

from endolas import LASTENSequence

import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as kb
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint

from endolas import UNet
from endolas import preprocess_input as pre_une

from matplotlib import pyplot as plt

# Checks
The version of tensorflow as well as the GPU support are checked.

In [2]:
print(tf.__version__)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.get_visible_devices('GPU')

2.2.0-rc3


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# 12) Supervised Euclidean for LASTEN2AK with Crossvalidation
A U-Net is trained here to predict a displacement field and the euclidean distance is taken as error.

# Data
Import training images

In [59]:
width = shape #Get through papermill
height = shape #Get through papermill

spacing_x = spacing #Get through papermill
spacing_y = spacing #Get through papermill

experiment = experiment #Get through papermill

grid_width = 18
grid_height = 18

store_path = '../results/12_{}_euclidean'.format(experiment)
data_path = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN2AK'

#store_path = '..\results\12_{}_euclidean'.format(experiment)
#data_path = r'D:\Julian\data\LASTEN2AK'

# Seeding
Seeds are set to ensure reproducible training.

In [4]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Functionality
Functionality to reinitalize the weights

In [39]:
def reinit_weights(model):
    for ix, layer in enumerate(model.layers):
        if hasattr(model.layers[ix], 'kernel_initializer') and hasattr(model.layers[ix], 'bias_initializer'):
            weight_initializer = model.layers[ix].kernel_initializer
            #bias_initializer = model.layers[ix].bias_initializer

            old_weights = model.layers[ix].get_weights()[0]
            #old_biases = model.layers[ix].get_weights()[1]
            
            model.layers[ix].set_weights([weight_initializer(shape=old_weights.shape)])#,bias_initializer(shape=len(old_biases))])

# Training Loop
Prepare the settings for training the model.

In [18]:
# Clear session
tf.keras.backend.clear_session()

# Training parameters
batch_size=4
folds = 5

# Get the model
model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(width, height, 2))

eu_loss = closs.EuclideanLoss(batch_size=batch_size, grid_width=grid_width, grid_height=grid_height, loss_type='msed')
eu_met = closs.EuclideanLoss(batch_size=batch_size, grid_width=grid_width,
                             grid_height=grid_height, loss_type='medn', grid_spacing=spacing)

model.compile(optimizer='adam', loss=eu_loss, metrics=[eu_met])

for fold in range(folds):
    # Reinit model
    reinit_weights(model)
    
    # Determine data paths
    folds = [0, 1, 2, 3, 4]
    folds.remove(fold)
    path_train = data_path + os.sep + str(fold) + os.sep + 'fold_{}_{}_{}_{}'.format(folds[0],
                                                                                     folds[1],
                                                                                     folds[2],
                                                                                     folds[3])
    path_validation = data_path + os.sep + str(fold) + os.sep + 'fold_{}'.format(fold)
    
    path_fixed = data_path + os.sep + 'fixed_images' + os.sep + '{}_{}'.format(width, spacing)
    
    # Get all training folds
    train_gen = LASTENSequence(path_train,
                               path_fixed,
                               batch_size=batch_size,
                               width=width,
                               height=height,
                               grid_width=grid_width, 
                               grid_height=grid_height,
                               preprocess_input=pre_une,
                               shuffle=True,
                               label="keypoints",
                               channel="moving+fixed")
    
    # Get validation fold
    val_gen = LASTENSequence(path_validation,
                             path_fixed,
                             batch_size=batch_size,
                             width=width,
                             height=height,
                             grid_width=grid_width, 
                             grid_height=grid_height,
                             preprocess_input=pre_une,
                             shuffle=False,
                             label="keypoints",
                             channel="moving+fixed")

    # Callbacks
    store_fold_path = store_path + os.sep + int(fold)
    
    logger = CSVLogger(store_fold_path + os.sep + 'log')
    timelogger = ccall.TimeHistory(store_fold_path)
    checker_best = ModelCheckpoint(store_fold_path + os.sep + "best_weights.hdf5", save_best_only=True)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=10, patience=5, mode='min')
    callbacks = [timelogger, logger, checker_best, early_stopping]

    
    # Train the model
    model.fit(train_gen,
              epochs=70,
              callbacks=callbacks,
              validation_data=val_gen,
              validation_freq=1)